# Notebook Goals:
- [] Remove major errors, duplicates, and outliers
- [x] Remove unwanted data field(s)
    * Removed all irrelevant field(s).
- [x] Bring structure
    * Parsed the Date into two columns.
- [x] Handle missing data
    * Filled all NA values with zeroes (0).

In [49]:
import pandas as pd

In [50]:
# Optimize loading the DataFrame into memory by only requesting the columns that are relevant towards the problem statement.
requested_columns = ["INCIDENT_DATE", "INCIDENT_NUMBER", "LOCATION_DISTRICT", "OFFENSE_DESCRIPTION", "WEAPON_TYPE", "INCIDENT_LOCATION", "ZIP", "LATITUDE", "LONGITUDE"]

lrpd = pd.read_csv("../data/lrpd.csv", usecols=requested_columns)

In [51]:
# Drop all duplicate rows!
prev_shape = lrpd.shape
lrpd = lrpd.drop_duplicates(subset=["INCIDENT_NUMBER"], keep='last')
new_shape = lrpd.shape

In [52]:
diff = prev_shape[0] - new_shape[0]
print(f'Dropped {diff} duplicates!')

Dropped 6369 duplicates!


In [53]:
lrpd.set_index("INCIDENT_NUMBER", inplace=True)

In [54]:
# Fill all the Pandas-handled Missing Data with zeroes (0).
lrpd = lrpd.fillna(0)

In [55]:
lrpd.head()

,INCIDENT_DATE,LOCATION_DISTRICT,OFFENSE_DESCRIPTION,WEAPON_TYPE,INCIDENT_LOCATION,ZIP,LATITUDE,LONGITUDE
INCIDENT_NUMBER,,,,,,,,
2021-092971,08/04/2021 10:21:00 AM,72.0,THEFT FROM MOTOR VEHICLE,0,15601 KANIS RD,72204.0,0.000000,0.000000
2017-029450,03/16/2017 06:30:00 PM,82.0,THEFT OF MOTOR VEHICLE PARTS,0,10801 IRONTON CUTOFF RD,72206.0,34.649362,-92.301959
2017-156453,09/01/2017 12:00:00 PM,53.0,RAPE,0,0,0.0,0.000000,0.000000
2019-130717,10/17/2019 09:00:00 PM,51.0,RAPE,0,0,0.0,0.000000,0.000000
2020-073313,07/02/2020 07:10:00 PM,50.0,RAPE,0,0,0.0,0.000000,0.000000


In [56]:
# Splits the incident_date string into a 3-element tuple.
# [0] -> MM/DD/YYYY
# [1] -> HH:MM:SS
# [2] -> AM/PM
def split_incident_date(incident_date):
    split = incident_date.split(' ')
    return split[0], split[1], split[2]

# Parses the split incident into a Date format (MM/DD/YYYY).
def get_incident_date(incident_date):
    date = split_incident_date(incident_date)
    return date[0]

# Parses the incident into a Time format (HH:MM AM/PM).
def get_incident_time(incident_date):
    time = split_incident_date(incident_date)
    hms = time[1].split(':')
    return hms[0] + ':' + hms[1] + ' ' + time[2]

In [57]:
# Replace INCIDENT_DATE column with more concise columns.
temp_date = lrpd["INCIDENT_DATE"]
lrpd["INCIDENT_DATE"] = temp_date.apply(get_incident_date)
lrpd["INCIDENT_TIME"] = temp_date.apply(get_incident_time)

In [58]:
lrpd.head()

,INCIDENT_DATE,LOCATION_DISTRICT,OFFENSE_DESCRIPTION,WEAPON_TYPE,INCIDENT_LOCATION,ZIP,LATITUDE,LONGITUDE,INCIDENT_TIME
INCIDENT_NUMBER,,,,,,,,,
2021-092971,08/04/2021,72.0,THEFT FROM MOTOR VEHICLE,0,15601 KANIS RD,72204.0,0.000000,0.000000,10:21 AM
2017-029450,03/16/2017,82.0,THEFT OF MOTOR VEHICLE PARTS,0,10801 IRONTON CUTOFF RD,72206.0,34.649362,-92.301959,06:30 PM
2017-156453,09/01/2017,53.0,RAPE,0,0,0.0,0.000000,0.000000,12:00 PM
2019-130717,10/17/2019,51.0,RAPE,0,0,0.0,0.000000,0.000000,09:00 PM
2020-073313,07/02/2020,50.0,RAPE,0,0,0.0,0.000000,0.000000,07:10 PM


In [59]:
# Handle outliers.
# Reference: https://hersanyagci.medium.com/detecting-and-handling-outliers-with-pandas-7adbfcd5cad8
lrpd.describe()

,LOCATION_DISTRICT,ZIP,LATITUDE,LONGITUDE
count,81833.000000,81833.000000,81833.000000,81833.000000
mean,65.955446,71271.371867,33.596511,-89.350141
std,15.510309,8160.766312,6.157083,16.374495
min,0.000000,0.000000,0.000000,-92.545466
25%,54.000000,72204.000000,34.689606,-92.387244
50%,63.000000,72206.000000,34.732894,-92.346238
75%,81.000000,72209.000000,34.752135,-92.314721
max,93.000000,72227.000000,34.881691,0.000000


In [60]:
Q1 = lrpd.quantile(0.25)
Q3 = lrpd.quantile(0.75)
IQR = Q3 - Q1

In [61]:
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

In [62]:
# Drop the outliers.
outliers_15_low = (lrpd < lower_limit)
outliers_15_up = (lrpd > upper_limit)

C:\Users\clayt\AppData\Local\Temp\ipykernel_14796\2720126428.py:2: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  outliers_15_low = (lrpd < lower_limit)
C:\Users\clayt\AppData\Local\Temp\ipykernel_14796\2720126428.py:3: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  outliers_15_up = (lrpd > upper_limit)


In [63]:
# Save the cleaned dataset.
lrpd.to_csv("../data/lrpd-clean.csv")